# Testing Regression vs. Galerkin Projection

This script tests the accuracy of regression at different PC orders, dimensions, and number of training points.

In [1]:
# Imports
import numpy as np
import math
from __future__ import print_function
 

import PyUQTk.pce as uqtkpce
import PyUQTk.PyPCE.pce_tools as pce_tools
from PyUQTk.utils.func import *

import sys
sys.path.append('../pyuqtkarray/')
import PyUQTk.uqtkarray as uqtkarray

import polynomial

import pandas as pd

PyMC is required for some of the MCMC postprocessing codes.
Will proceed without, but some convergence tests will not be available.


## Inputs

In [2]:
nord = 4            # Order of the PCE
pc_type = "LU"      # Polynomial type
pc_alpha = 0.0      # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs
pc_beta = 1.0       # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs
param = nord+1      # Number of quadrature points per dimension for full quadrature or level for sparse quadrature
ndim = 5            # Number of dimensions

We use a pc set object to hold the polynomial we will use. We define our coefficients as a sequence from 1 to 10.

In [3]:
poly = uqtkpce.PCSet("NISPnoq", nord, ndim,pc_type, pc_alpha,pc_beta)
npce=poly.GetNumberPCTerms()
coef=np.array([i+1 for i in range(npce)])

We use random testing points in [-1,1].

In [4]:
nTest=npce
np.random.seed(42)
rng = np.random.default_rng()
rand=2*rng.random((int(nTest), ndim))-1

In order to test regression, we are creating a matrix
Ax=b
A is the evaluated basis
where x are the coefficients,
b are the evaluations of the polynomial.

We create b, which is an array of evaluations of polynomial chaos expansion (a linear combination of the chosen polynomial types, using the chosen coefficients).

In [5]:
# Creating b
b = pce_tools.UQTkEvaluatePCE(poly, coef, rand)

We use regression to create an array of basis evaluations A and find x, our coefficients. If done correctly, x should be the array we originally chose.

In [6]:
# Regression creates A and finds x
c_k=pce_tools.UQTkRegression(poly, b, rand)

Was the regression test successful? Select a decimal place to compare answers to

In [7]:
dec_place=5
reg=(np.round(c_k, dec_place)==coef)
(all(reg) and reg[0]==True)

True

Galerkin projection should produce the original coefficients as well.

In [8]:
#Galerkin for comparison
pc_model = uqtkpce.PCSet("NISPnoq", nord, ndim,pc_type, pc_alpha,pc_beta)
coef=np.array([i+1 for i in range(npce)])

pc_model.SetQuadRule(pc_type, 'full', param)
qdpts, totquat= pce_tools.UQTkGetQuadPoints(pc_model)
f_evals=pce_tools.UQTkEvaluatePCE(pc_model, coef, qdpts)

c_k2 = pce_tools.UQTkGalerkinProjection(pc_model,f_evals)

In [9]:
pd.DataFrame(data=np.transpose([c_k, c_k2]), columns=['Regression', "Galerkin"])

,Regression,Galerkin
0,1.0,1.0
1,2.0,2.0
2,3.0,3.0
3,4.0,4.0
4,5.0,5.0
5,6.0,6.0
6,7.0,7.0
7,8.0,8.0
8,9.0,9.0
9,10.0,10.0
